# Analyze Commit Messages

## Imports

In [1]:
import pandas as pd
from scipy import spatial
from functools import reduce
from pydriller import RepositoryMining
import json
import string
import nltk
from collections import Counter

In [2]:
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

## Loading Raw Dataset

In [3]:
# Read file
df = pd.read_excel("../../data/B_Manual_Commit_Analysis/commit_analyis.xlsx", index_col=0)

In [4]:
# Restore lists, Convert import sets to binary feature vectors
df["imports"] = df["imports"].apply(eval).apply(set)
df["keywords"] = df["keywords"].apply(eval).apply(set)

In [5]:
df.to_pickle("commit_analysis.pkl")

## Loading Dump

In [6]:
df = pd.read_pickle("commit_analysis.pkl")

## Collect Commit Messages

In [7]:
commit_msgs_dict = {}
with open("../../data/B_Manual_Commit_Analysis/commit_messages.json") as json_file:
    commit_msgs_dict = json.load(json_file)

In [8]:
def getMessageFromDict(short_commit, repo_link, commit_msgs_dict):
    #print("Analyzing {:<8} repository {:<150}".format(short_commit,repo_link),end="\r")
    pre_selected_keys = [k for k in commit_msgs_dict.keys() if k.startswith(short_commit)  and repo_link.endswith(k.split("___")[1]+".git")]
    if(len(pre_selected_keys)<=0):
        print("Error Did not found any commit for short commit {}. Will return empty message!".format(short_commit))
        return ""
    if(len(pre_selected_keys)>1):
        print("Warning: There exist multiple entries for short commit {}. Will return first message! These are all messages:".format(short_commit))
        for key in pre_selected_keys:
            print("\t'{}'".format(commit_msgs_dict[key]))
    return commit_msgs_dict[pre_selected_keys[0]]

In [9]:
df["commit_message"] = df.apply(lambda x: getMessageFromDict(x['commit'], x['repo'],commit_msgs_dict), axis=1)

In [10]:
df.to_pickle("commit_analysis_messages.pkl")

In [11]:
df = pd.read_pickle("commit_analysis_messages.pkl")

## Filter Dataset

In [12]:
def getSubSetWithPrefix(setValue, prefixString):
    return {s for s in setValue if s.startswith(prefixString)}

### Get number changed methods

In [13]:
countOccurences = df.groupby("commit")["commit"].count()
df["occurrences"] = df["commit"].apply(lambda x: countOccurences[x])

### Filter for Android imports and changed methods

In [14]:
df["android_imports"] = df["imports"].apply(lambda x : getSubSetWithPrefix(x,"android."))
df_filtered = df.loc[(df["android_imports"]!=set()) & (df["occurrences"]<=10)]

In [15]:
print("{} apps and in sum {} small (<=10 changed methods) commits with android-specific fix.".format(len(df_filtered["app"].unique()),len(df_filtered["commit"].unique())))

3 apps and in sum 29 small (<=10 changed methods) commits with android-specific fix.


## Natural Language Processing of Commit Messages

In [16]:
df_single_commits = df[["repo","commit","commit_message"]].drop_duplicates()

### Preprocessing

In [17]:
stop_words = nltk.corpus.stopwords.words('english')
def preprocessing(text):
    # inspired by https://towardsdatascience.com/nlp-preprocessing-with-nltk-3c04ee00edc0
    
    # lowercase
    tmp_txt = text.lower()
    
    # remove punctutation
    tmp_txt = "".join([char for char in tmp_txt if char not in string.punctuation])
    
    # tokenize
    tokens = nltk.word_tokenize(tmp_txt) 
    
    # remove english stop words
    tokens = [token for token in tokens if token not in stop_words]
    
    #stemming
    #porter = nltk.stem.porter.PorterStemmer()
    #tokens = [porter.stem(token) for token in tokens]
    
    return tokens

In [18]:
df_single_commits["prep_commit_message"] = [preprocessing(text) for text in df_single_commits["commit_message"]]
#remove commit message from dataset
df_single_commits = df_single_commits.drop(columns=["commit_message"])

### Filter via Commit Messages

In [19]:
# android versions based on https://en.wikipedia.org/wiki/Android_version_history
# since version 10 version naming is 'Android <version-number>' w.r.t. https://www.republicworld.com/technology-news/mobile/all-the-android-versions-named-by-google-alphabetically.html
android_specific_keywords = {"android","api level",
                              "cupcake",#1.5
                              "donut",#1.6
                              "eclair",#2.0-2.1
                              "froyo",#2.2-2.2.3
                              "gingerbread",#2.3-2.3.7
                              "honeycomb",#3.0-3.26
                              "ice cream sandwich",#4.0-4.0.4
                              "jelly bean",#4.1-4.3.1
                              "kitkat",#4.4-4.4.4
                              "lollipop",#5.0-5.1.1
                              "marshmallow",#6.0-6.0.1 
                              "nougat",#7.0-7.1.2
                              "oreo",#8.0-8.1
                              "pie"#9
                            }

In [20]:
def contains_android_keywords(tokens):
    return any([token in android_specific_keywords for token in tokens])

In [21]:
df_single_commits["has_android_msg"] = df_single_commits["prep_commit_message"].apply(lambda x: contains_android_keywords(x))
df_commit_msgs_with_android = df_single_commits[df_single_commits["has_android_msg"]].replace(['\r','\n'],' ', regex=True)
#add columns for manual assessment
df_commit_msgs_with_android["is_android_specific_misuse"] = ""
df_commit_msgs_with_android["notes"] = ""
df_commit_msgs_with_android["is_android_version-compat_check"] = ""
df_commit_msgs_with_android.to_excel("../../data/B_Manual_Commit_Analysis/commits_android_messages.xlsx")